# OpenStreetMap Case Study


<h2>Map Area - San Francisco, United States</h2>
<p>I picked up this city becuase this is one of my most favorate and beautiful city and I belive my case study will help to improve the OpenStreetMap.org data<p>

## Problems Encountered in the Map
<p>After initial downloading a small sample size of the San Francisco area and running it against the above code, I noticed few main problems with the data, which I will discuss in the following order:</p>

* The telephone number is stored in different format across the file.
* Street names are abbreviated in few occurrence but not in other instances. 
* State name is not stored consistently across the file

#### Sampling
<p>After downloading my large data set, I would like to peform sampling for the following fields to check the data integerity & data consitance issues.</p>
<p>Since this is an case study I opted to perform analysis only for the following fields. </p>

* phone
* street
* state

<p>Note: Ideal way to audit the data is to load the sample data in SQLite DB and analyse the data, but for our case study I preferred to analyse the data in Notebook. </p>

I have used "OSM_submission_file.py" file to clean my data & generate the new csv files.

<h1>Importing data into SQLITE </h1>
<p>I have generated 5 CSV files to store in the xml data in the database. While generating the CSV files I have considered the relationship between each tables such that I can join two tables based on its key values</p>

<p>Now its time export the data into local database. In SQLlite terminal, I have executed the following commands to create the tables & import the data from the csv files</p>
<li>SQLITE3 OSM.DB</li>
<li>.mode csv</li>
<li>.import filename.csv tablename</li>

<p>Now I have created the DB & imported the csv files sucessfully</p>


#### Connect the DB & Explore the data

In [3]:
#This function helps to connect SQLLITE DB 
def sqlite(query):
    conn=sqlite3.connect('OSM.db')
    df=pd.read_sql_query(query,conn) #Reading the storing the data into dataframe
    return df

## Checking Phone Numbers, Street & State values

<p>I'm checking the three fields that I have fixed the data issue during my intial analysis. I am expecting to see properly formatted phone numbers, street & state information across all the tables. </p>

#### Querying Street value
<p>I am querying the table to find the invalid street names and the SQL should return zero counts. </p>

In [269]:
sqlite('select count(*) from ways_tags where key="addr:street" and lower(value) like "% st" union all select count(*) from ways_tags where key="address" and lower(value) like "% st" ')

,count(*)
0,0
1,0


#### Querying Phone number
<p> This query checks if there are any phone numbers that are stored in different format. </p>

In [14]:
sqlite('select count(*) from nodes_tags where key ="phone"  and (length(value)<12 or length(value)>12) union all select count(*) from ways_tags where key ="phone"  and (length(value)<12 or length(value)>12) ')

,count(*)
0,6
1,2


I have noticed there are 8 invalid phone numbers found in our database but rest of the entries are stored as expected. 

# Data Overview and Additional Data Exploration

<p>This section contains basic statistics about the dataset, the SQL queries used to gather them, 
and some additional ideas about the data in context.</p>

### File sizes

In [248]:
# This code prints the file size
OSM_XML=os.stat('OSM.XML')
Nodes_csv=os.stat('nodes.csv')
Nodes_tags=os.stat('nodes_tags.csv')
Ways=os.stat('ways.csv')
Ways_tags=os.stat('ways_tags.csv')
Ways_nodes=os.stat('ways_nodes.csv')

print("OSM.XML---->" + str(OSM_XML.st_size >> 20)+" MB")
print("NODES.CSV---->" + str(Nodes_csv.st_size >> 20)+" MB")
print("NODES_WAYS.CSV---->" + str(Nodes_tags.st_size >> 20)+" MB")
print("WAYS.CSV---->" + str(Ways.st_size >> 20)+" MB")
print("WAYS_TAGS.CSV---->" + str(Ways_tags.st_size >> 20)+" MB")
print("WAYS_NODES.CSV---->" + str(Ways_nodes.st_size >> 20)+" MB")

OSM.XML---->106 MB
NODES.CSV---->39 MB
NODES_WAYS.CSV---->2 MB
WAYS.CSV---->3 MB
WAYS_TAGS.CSV---->4 MB
WAYS_NODES.CSV---->14 MB


### Number of nodes

In [194]:
sqlite('SELECT COUNT(*) FROM nodes')

,COUNT(*)
0,2373


### Number of ways

In [196]:
sqlite('SELECT COUNT(*) FROM ways')

,COUNT(*)
0,552


### Number of unique users

In [200]:
sqlite('SELECT COUNT(DISTINCT(e.uid)) Unique_user FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e')

,Unique_user
0,147


### Top 10 Amenity

In [30]:
sqlite('select value, Total from (select value, count(*) as "Total" from nodes_tags where key="amenity" group by value union all select value, count(*) as "Total" from ways_tags where key="amenity" group by value) order by Total desc limit 10 ')

,value,Total
0,restaurant,2044
1,cafe,712
2,post_box,622
3,bicycle_parking,446
4,bench,434
5,parking,363
6,bar,324
7,fast_food,268
8,pub,230
9,car_sharing,224


## Additional Data Exploration

#### Total Number of Subways

In [38]:
sqlite('select count(*) "#ofSubway" from nodes_tags where key="subway"')

,#ofSubway
0,28


#### Most popular resturant

In [36]:
sqlite('SELECT nodes_tags.value, COUNT(*) as num FROM nodes_tags JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value="restaurant") i ON nodes_tags.id=i.id WHERE nodes_tags.key="cuisine" GROUP BY nodes_tags.value ORDER BY num DESC limit 10')

,value,num
0,mexican,118
1,italian,108
2,chinese,96
3,japanese,86
4,thai,82
5,pizza,80
6,american,76
7,vietnamese,66
8,indian,50
9,sushi,44


## Other ideas about the dataset


One of the suggestions I would like to make is that if OpenStreetMap contains user rating information like Yelp, one can create a metrics easily and help customer & Business owners.

###### Benifits
* It helps customer/user to make decision/choice easily
* It helps business owners to improve their business.
* User can create reports like a) Top Rated restaurant by Category. 

##### Anticipated problems

* Since OpenStreetMap doesn't interact directly with user and collecting & storing user rating from the scratch  would be a tedious job
* The other approach I can think of is to get this data from Yelp, but there are challenges to match our data with Yelp since there is no primary key and the foreign key relationship defined between these two databases. 

## Conclusion

<p>After this review of the data it’s obvious that the San Francisco area is incomplete, though I believe it has been well cleaned for the purposes of this exercise.I think it would be possible to input a great amount of cleaned data to OpenStreetMap.org.</p>